**Importing relevant modules**

In [ ]:
from textblob import TextBlob
import unicodedata as uni
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
#from sklearn import cross_validation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.stem.wordnet import WordNetLemmatizer 
#stemmer = WordNetLemmatizer()
import csv
from scipy.stats import spearmanr
import sklearn.feature_extraction.text as text
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import string

import unicodedata as uni
!pip install emoji
import emoji

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords



import math
import gensim.models as gs
import pickle as pk
import sklearn.metrics as met
import scipy.stats as stats
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Importing the Datasets**

In [ ]:
path1 = "Final_Emojis - emotions1.csv"
df1 = pd.read_csv(path1)
df1.keys()

path2 = "Dataset_With_Emoji.csv"
df2 = pd.read_csv(path2)
df2.keys()

Index(['Unnamed: 0', 'Tweet', 'Affect Dimension', 'Unnamed: 2'], dtype='object')

**Visualize first dataset**

In [ ]:
df1.head()

,Tweet,Affect Dimension,Unnamed: 2
0,😡@chetan_bhagat 😤😤 Right to speech... Its bitt...,anger,NaN
1,Low key Dro f*cked Molly like he had a point t...,anger,NaN
2,@TeleTransOne @eleganza_lily @FemFataleXX Tha...,anger,NaN
3,@F3Alpha #F3Roswell #Preblast\nI've 😤😤been han...,anger,NaN
4,Did they offend us and they want it to sound n...,anger,NaN


**Visualize second dataset**

In [ ]:
df2.head()

,Unnamed: 0,Tweet,Affect Dimension,Unnamed: 2
0,0,pouting face @chetan_bhagat face with steam ...,anger,NaN
1,1,Low key Dro f*cked Molly like he had a point t...,anger,NaN
2,2,@TeleTransOne @eleganza_lily @FemFataleXX Tha...,anger,NaN
3,3,@F3Alpha #F3Roswell #Preblast\nI've face with...,anger,NaN
4,4,Did they offend us and they want it to sound n...,anger,NaN


In [ ]:
#In case we want to eliminate Neutral statements
#df1 = df1.drop(df1[df1['Affect Dimension']=='neutral'].index, axis = 0)

In [ ]:
#Code used to convert Emojis to Text
"""def is_emoji(s):
    count = 0
    for emoji1 in emoji.UNICODE_EMOJI:
        count += s.count(emoji1)
        if count > 1:
            return False
    return bool(count)

def emoji2text(title):
    stemmed = []
    tokenized = []
    series = []
    print(type(series))
    for i in title:
      #print(type(title))
      for c in i:
        if is_emoji(c) == True:
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', ' ')
          #word2 = " ".join(word1)
          i = i.replace(c, word1)
      series.append(i)
    return series

df1.Tweet = emoji2text(df1.Tweet)
tweets = (df1['Tweet'])
df1.head()
labels = df1['Affect Dimension']
df1.to_csv('Dataset_With_Emoji.csv')"""

'def is_emoji(s):\n    count = 0\n    for emoji1 in emoji.UNICODE_EMOJI:\n        count += s.count(emoji1)\n        if count > 1:\n            return False\n    return bool(count)\n\ndef emoji2text(title):\n    stemmed = []\n    tokenized = []\n    series = []\n    print(type(series))\n    for i in title:\n      #print(type(title))\n      for c in i:\n        if is_emoji(c) == True:\n          word1 = emoji.demojize(c)\n          word1 = word1.replace(\'_\', \' \')\n          word1 = word1.replace(\':\', \' \')\n          #word2 = " ".join(word1)\n          i = i.replace(c, word1)\n      series.append(i)\n    return series\n\ndf1.Tweet = emoji2text(df1.Tweet)\ntweets = (df1[\'Tweet\'])\ndf1.head()\nlabels = df1[\'Affect Dimension\']\ndf1.to_csv(\'Dataset_With_Emoji.csv\')'

**Tokenization if emojis in dataset are not converted to text, and we need to take text into consideration (takes a long time to run, which is why we already converted emojis in our dataset for repeated use)**

In [ ]:
#main function
"""def ts(title):
    stemmed = []
    tokenized = []
    for i1, i in enumerate(title):
        print(i1)
        stemmed1 = tokenstem(i) 
        tokenized1 = token(i)
        stemmed.extend(stemmed1)
        tokenized.extend(tokenized1)
    return stemmed, tokenized

#side functions
def tokenstem(text):
    words1 = []
    tk = nltk.TweetTokenizer()
    #words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)] #tokenize sentences then word
    try:
      words = [word for sent in nltk.sent_tokenize(text) for word in tk.tokenize(sent)]
    except:
      print(text)
    
    #print(words)
    words11=words.copy()
    idx=0
    i2=0
    n=0
    #print(words11)
    for i1, word in enumerate(words11):
      
      #print(word)
      if i2!=0 and n!=0:
        words.insert(idx, word1)
        idx+=1
        n=0
      if i2==0 and i1!=0:
        idx+=1
      i2=0
      m=0
      n=0
      #print(idx)
      for c in word:
      #c for c in str if c in emoji.UNICODE_EMOJI
        #print(c)
        if is_emoji(c) == True:
          i2+=1
          if m==0:
            try:
              del(words[idx])
            except:
              print(words, idx)
          if n!=0:
            words.insert(idx, word1)
            idx+=1
            n=0
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', '')
          word1 = nltk.word_tokenize((word1))
          for i in range(len(word1)):
            words.insert(idx, word1[i])
            idx+=1
          m+=1
        else:
          if n==0:
            word1 = c
          else:
            word1=word1+c
          n+=1
    for token in words:
        if re.search('[a-zA-Z]', token): #check if it is a word
            words1.append(token)
    #stems = [stemmer.lemmatize(t) for t in words1]
    stems = [stemmer.stem(t) for t in words1]
    return stems

def token(text):
    words2 = []
    tk = nltk.TweetTokenizer()
    #words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    try:
      words = [word for sent in nltk.sent_tokenize(text) for word in tk.tokenize(sent)]
    except:
      print(text)
    words11=words.copy()
    idx=0
    i2=0
    n=0
    for i1, word in enumerate(words11):
      #print(word)
      if i2!=0 and n!=0:
        words.insert(idx, word1)
        idx+=1
        n=0
      if i2==0 and i1!=0:
        idx+=1
      i2=0
      m=0
      n=0


      for c in word:
      #c for c in str if c in emoji.UNICODE_EMOJI
        if is_emoji(c) == True:
          i2+=1
          if m==0:
            try:
              del(words[idx])
            except:
              print(words, idx)
          if n!=0:
            words.insert(idx, word1)
            idx+=1
            n=0
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', '')
          word1 = nltk.word_tokenize((word1))
          for i in range(len(word1)):
            words.insert(idx, word1[i])
            #print(i)
            idx+=1
          m+=1
        else:
          if n==0:
            word1 = c
          else:
            word1=word1+c
          n+=1
    for token in words:
        if re.search('[a-zA-Z]', token):
            words2.append(token)
    return words2"""

"def ts(title):\n    stemmed = []\n    tokenized = []\n    for i1, i in enumerate(title):\n        print(i1)\n        stemmed1 = tokenstem(i) \n        tokenized1 = token(i)\n        stemmed.extend(stemmed1)\n        tokenized.extend(tokenized1)\n    return stemmed, tokenized\n\n#side functions\ndef tokenstem(text):\n    words1 = []\n    tk = nltk.TweetTokenizer()\n    #words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)] #tokenize sentences then word\n    try:\n      words = [word for sent in nltk.sent_tokenize(text) for word in tk.tokenize(sent)]\n    except:\n      print(text)\n    \n    #print(words)\n    words11=words.copy()\n    idx=0\n    i2=0\n    n=0\n    #print(words11)\n    for i1, word in enumerate(words11):\n      \n      #print(word)\n      if i2!=0 and n!=0:\n        words.insert(idx, word1)\n        idx+=1\n        n=0\n      if i2==0 and i1!=0:\n        idx+=1\n      i2=0\n      m=0\n      n=0\n      #print(idx)\n      for c in word:

**For tokenization and stemming only**

In [ ]:
def ts(title):
    stemmed = []
    tokenized = []
    for i in title:
        stemmed1 = tokenstem(i) 
        tokenized1 = token(i)
        stemmed.extend(stemmed1)
        tokenized.extend(tokenized1)
    return stemmed, tokenized

#side functions
def tokenstem(text):
    words1 = []
    tk = nltk.TweetTokenizer()
    #words = [word for sent in sent1 for word in nltk.word_tokenize(sent)] #tokenize sentences then word
    try:
      words = [word for sent in nltk.sent_tokenize(text) for word in tk.tokenize(sent)]
    except:
      print(words, text)
    #words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)] #tokenize sentences then word
    for token in words:
        if re.search('[a-zA-Z]', token): #check if it is a word
            words1.append(token)
    #stems = [stemmer.lemmatize(t) for t in words1]
    stems = [stemmer.stem(t) for t in words1]
    return stems

def token(text):
    words2 = []
    tk = nltk.TweetTokenizer()
    #words = [word for sent in sent1 for word in nltk.word_tokenize(sent)] #tokenize sentences then word
    words = [word for sent in nltk.sent_tokenize(text) for word in tk.tokenize(sent)]
    #words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)] #tokenize sentences then word
    for token in words:
        if re.search('[a-zA-Z]', token):
            words2.append(token)
    return words2

Using TFIDF Vectorizer

In [ ]:
#On Dataset 1
titles = df1.Tweet.str.lower() #to make it lower case
stemmedop, tokenizedop = ts(titles)
#To remove repitions for better output
words = zip(stemmedop, tokenizedop) 
words = list(set(words))
stemmed2, tokenized2 = zip(*words)
merged = pd.DataFrame({'words': tokenized2}, index = stemmed2)  #to put words under a specific stem
#Using NLTK to get stopwords to remove it from our list
stopwords = nltk.corpus.stopwords.words('english')
stop_words = text.ENGLISH_STOP_WORDS.union(stopwords)
# tf-idf vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df =10**-3 ,analyzer = 'word', max_features=len(set(stemmed2)), stop_words=stop_words, tokenizer=tokenstem, ngram_range=(1,3))
tfidf1 = tfidf_vectorizer.fit_transform(titles)

#On Dataset 2
titles = df2.Tweet.str.lower() #to make it lower case
stemmedop, tokenizedop = ts(titles)
#To remove repitions for better output
words = zip(stemmedop, tokenizedop) 
words = list(set(words))
stemmed2, tokenized2 = zip(*words)
merged = pd.DataFrame({'words': tokenized2}, index = stemmed2)  #to put words under a specific stem
#Using NLTK to get stopwords to remove it from our list
stopwords = nltk.corpus.stopwords.words('english')
stop_words = text.ENGLISH_STOP_WORDS.union(stopwords)
# tf-idf vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df =10**-3 ,analyzer = 'word', max_features=len(set(stemmed2)), stop_words=stop_words, tokenizer=tokenstem, ngram_range=(1,3))
tfidf2 = tfidf_vectorizer.fit_transform(titles)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'doe', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', "should'v", 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local

**Initializing Stratified K-Fold Cross Validation**

In [ ]:
a=np.array(df1['Affect Dimension'])
kf = StratifiedKFold(n_splits=10, shuffle=True)

**Applying Linear SVM**

In [ ]:
#Dataset1
clf = LinearSVC()
scores=(cross_val_score(clf, tfidf1, a, cv=kf))
print("Accuracy without Emoji: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

#Dataset2
clf = LinearSVC()
scores=(cross_val_score(clf, tfidf2, a, cv=kf))
print("Accuracy with Emoji: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

Accuracy without Emoji: 0.682812 (+/- 0.0250879)
Accuracy with Emoji: 0.830136 (+/- 0.0107389)


**Applying Logistic Regression**

In [ ]:
#Dataset 1
clf = LogisticRegression(max_iter=1000)
scores=(cross_val_score(clf, tfidf1, a, cv=kf, n_jobs=-1))
print("Accuracy without Emoji: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

#Dataset 2
clf = LogisticRegression(max_iter=1000)
scores=(cross_val_score(clf, tfidf2, a, cv=kf, n_jobs=-1))
print("Accuracy with Emoji: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

Accuracy without Emoji: 0.67751 (+/- 0.0228729)
Accuracy with Emoji: 0.834425 (+/- 0.0260759)


**Applying Multinomial Naïve Bias**

In [ ]:
#Dataset 1
clf = MultinomialNB()
scores=(cross_val_score(clf, tfidf1, a, cv=kf, n_jobs=-1))
print("Accuracy: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

#Dataset 2
clf = MultinomialNB()
scores=(cross_val_score(clf, tfidf2, a, cv=kf, n_jobs=-1))
print("Accuracy: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.673997 (+/- 0.0151847)
Accuracy: 0.834037 (+/- 0.0189582)


**Applying Random Forest Classifier**

In [ ]:
#Dataset 1
clf = RandomForestClassifier(n_jobs = -1, max_features = 'sqrt', n_estimators = 35, oob_score = True)
scores=(cross_val_score(clf, tfidf1, a, cv=kf))
print("Accuracy: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

#Dataset 2
clf = RandomForestClassifier(n_jobs = -1, max_features = 'sqrt', n_estimators = 35, oob_score = True)
scores=(cross_val_score(clf, tfidf2, a, cv=kf))
print("Accuracy: %g (+/- %g)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.658086 (+/- 0.0264358)
Accuracy: 0.817814 (+/- 0.0232699)
